<a href="https://colab.research.google.com/github/ahmedksultan/amst2001/blob/main/AMST2001_ws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
buildings_df = pd.read_csv("buildings.csv")

buildings_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'buildings.csv'

## Webscraping ##

In [12]:
buildings_url = "https://en.wikipedia.org/wiki/List_of_Cornell_University_buildings#cite_note-1"
buildings_result = requests.get(buildings_url)
print(buildings_result)

<Response [200]>


In [19]:
with open("buildings.html", "w") as writer:
  writer.write(buildings_result.text)

with open("buildings.html", "r") as reader:
  html_source = reader.read()

page = BeautifulSoup(html_source, "html.parser")

In [177]:
name_list = []
image_list = []
architect_list = []
built_list = []
location_list = []
notes_list = []

entries = page.find_all("td")

for i in range(0, 169*7+1, 7):
  if len(entries[i].text.replace("\n", "")) == 0:
    name_list.append("N/A")
  else:
    name_list.append(entries[i].text.replace("\n", ""))

for i in range(1, 169*7+2, 7):
  for item in entries[i]:
    if item.name == "span":
      for subitem in item:
        if subitem.name == "a":
          image_list.append("en.m.wikipedia.org" + subitem.get("href"))
    else:
      image_list.append("N/A")

for i in range(2, 169*7+3, 7):
  if len(entries[i].text.replace("\n", "")) == 0:
    architect_list.append("N/A")
  else:
    architect_list.append(entries[i].text.replace("\n", ""))

for i in range(3, 169*7+4, 7):
  if len(entries[i].text.replace("\n", "")) == 0:
    built_list.append("N/A")
  else:
    built_list.append(entries[i].text.replace("\n", "")[:4])

for i in range(4, 169*7+5, 7):
  if len(entries[i].text.replace("\n", "")) == 0:
    location_list.append("N/A")
  else:
    location_list.append(entries[i].text.replace("\n", ""))

for i in range(5, 169*7+6, 7):
  if len(entries[i].text.replace("\n", "")) == 0:
    notes_list.append("N/A")
  else:
    notes_list.append(entries[i].text.replace("\n", ""))

In [178]:
wiki_buildings_df = pd.DataFrame(data = {"Name": name_list,
                                         #"Image": image_list,
                                         "Architect": architect_list,
                                         "Year": built_list,
                                         "Location": location_list,
                                         "Notes" : notes_list})


wiki_buildings_df.tail()

,Name,Architect,Year,Location,Notes
165,Ganędagǫ: Hall,ikon.5 Architects,2021,121 Triphammer Road,"190,784 sq ft; 577 beds; the name honors the C..."
166,Toni Morrison Hall,ikon.5 Architects,2021,18 Sisson Place,"178,869 sq ft; 314 beds;Building includes Morr..."
167,Barbara McClintock Hall,ikon.5 Architects,2022,224 Cradit Farm Dr,"103,835 sq ft; 311 beds"
168,Hu Shih Hall,ikon.5 Architects,2022,141 Program House Dr,"138,773 sq ft; 429 beds"
169,Ruth Bader Ginsburg Hall,ikon.5 Architects,2022,155 Program House Dr,"162,849 sq ft; 504 beds"


In [181]:
wiki_buildings_df.head()

,Name,Architect,Year,Location,Notes
0,Andrew Dickson White House,William Henry Miller (B.Arch. 1872),1874,Central Campus,Begun by Miller and finished by Babcock. Built...
1,Anabel Taylor Hall,"Starrett, Van Vleck & Marsh (Ernest A. Van Vle...",1953,548 College Avenue,Inside is a memorial to alumni killed in World...
2,Atkinson Hall,Lake Flato Architects,Summ,Tower Road,"Four-story building; 90,000 square feet; will ..."
3,Bailey Hall,Green and Wicks,1912,Central Campus,Bailey Hall is the largest auditorium at Corne...
4,Baker Laboratory,Arthur N. Gibb (B.Arch 1890),1921,Central Campus,Donated by and named for banker George Fisher ...


In [182]:
buildings_df.head()

,Name,Category,ImageURL,Address,Latitude,Longitude,Notes,AKA
0,104 West! (Kosher Dining Hall),Co-ops,NaN,NaN,42.444311,-76.487879,Dining (www.campuslife.cornell.edu/campuslife/...,NaN
1,112 Edgemoor,Co-ops,http://images.localist.com/images/main/busines...,NaN,42.444091,-76.488412,Housing,NaN
2,120 Maple Ave.,Near Campus,NaN,NaN,42.441336,-76.475530,CIT,NaN
3,301 College Ave.,Collegetown,http://images.localist.com/images/main/busines...,NaN,42.440338,-76.484999,NaN,NaN
4,309 College Ave.,Collegetown,NaN,NaN,42.440817,-76.484976,The Nines,NaN


In [185]:
print(type(wiki_buildings_df["Name"][0]), type(buildings_df["Name"][0]))

<class 'str'> <class 'str'>


In [187]:
wiki_buildings_df.Name.astype(str)
buildings_df.Name.astype(str)

0                         104 West! (Kosher Dining Hall)
1                                           112 Edgemoor
2                                         120 Maple Ave.
3                                       301 College Ave.
4                                       309 College Ave.
                             ...                        
436                        Wilson Synchrotron Lab & Ring
437                                            Wing Hall
438                                        Winter Garden
439                                             Zeta Psi
440    Zucker Shrub Collection and Harris Lilac Colle...
Name: Name, Length: 441, dtype: object

In [191]:
tot_buildings_df = buildings_df.merge(wiki_buildings_df, on="Name")
tot_buildings_df.shape

(95, 12)

In [193]:
print("Total Data Loss: " + str((1-tot_buildings_df.shape[0]/buildings_df.shape[0])*100) + "%")

Total Data Loss: 78.45804988662131%
